In [188]:
import random
import math

#### The Perceptron

In [ ]:
class Perceptron:
    def __init__(self, input_size: int, learning_rate: float, activation_function:dict):
        self.input_size    = input_size
        self.learning_rate = learning_rate
        
        self.weights    = [random.uniform(-1, 1) for _ in range(input_size)]
        self.bias       = random.uniform(-1, 1)
        self.activation = activation_function
    
    def train(self, inputs, targets, epochs,verbose=False):    
        if len(inputs) != len(targets):
            raise ValueError('Inputs and targets must be of the same length')
        
        for _ in range(epochs):
            for input_vector, target in zip(inputs, targets):
                output              = self.forward_pass(input_vector)
                weighted_sum        = output['weighted_sum']
                activated_output    = output['activation']
                activation_gradient = self.activation['activation_diff'](weighted_sum)
                loss                = (activated_output - target[0]) ** 2

                if verbose:
                    print(f"Loss: {loss:.7f}, Weights: {self.weights}")

                # Update weights
                for i in range(self.input_size):
                    self.weights[i] -= self.learning_rate * (input_vector[i] * activation_gradient * (activated_output - target[0]))
                
                # Update bias
                self.bias -= self.learning_rate * (activation_gradient * (activated_output - target[0]))
            
        print(f"Final loss: {loss:.7f}, Weights: {self.weights}")
    
    def forward_pass(self, input_vector):
        if len(input_vector) != self.input_size:
            raise ValueError('Input vector size does not match expected input size')
        
        weighted_sum = sum(x * w for x, w in zip(input_vector, self.weights)) + self.bias
        activated_output = self.activation['activation'](weighted_sum)
        return {'weighted_sum': weighted_sum, 'activation': activated_output}

#### Activation Functions along with their derivative versions

In [248]:
def sigmoid(x): return 1 / (1 + math.exp(-x))
def sigmoid_diff(x): return sigmoid(x)*(1-sigmoid(x))

def linear(x): return x
def linear_diff(x): return 1

def relu (x): return max(0,x)
def relu_diff(x): return 0 if x<=0 else 1

#### Example 1 : Linear Function 2x+10

In [249]:
# Data

X = [[0],[1],[2],[3],[4],[5],[6],[7]]
Y = [[10],[12],[14],[16],[18],[20],[22],[24]]

In [250]:
# Initialize

neuron = Perceptron(1,0.05,{'activation':linear,'activation_diff':linear_diff})

In [252]:
neuron.forward_pass([10])['activation']  # Wrong Answer :(

-8.185070969389082

In [253]:
# Train for 200 epochs

neuron.train(X,Y,200,True)

Loss: 100.9247523, Weights: [-0.8138939760267934]
Loss: 152.7132096, Weights: [-0.8138939760267934]
Loss: 177.3679202, Weights: [-0.19600804479219913]
Loss: 117.7806343, Weights: [1.1357874359092905]
Loss: 21.7403059, Weights: [2.763689236312073]
Loss: 0.9936536, Weights: [3.696219255984846]
Loss: 1.3178289, Weights: [3.447013820677279]
Loss: 0.0160913, Weights: [3.1026236311848203]
Loss: 57.7274243, Weights: [3.0582256315168204]
Loss: 37.9423735, Weights: [3.0582256315168204]
Loss: 17.4519387, Weights: [3.366212536638451]
Loss: 1.8203301, Weights: [3.7839677121935407]
Loss: 1.7206856, Weights: [3.986347127863985]
Loss: 3.6893177, Weights: [3.723997312738151]
Loss: 0.4456624, Weights: [3.24380739421752]
Loss: 0.2266626, Weights: [3.0435335466196904]
Loss: 46.9560490, Weights: [2.8769017041706473]
Loss: 31.7298359, Weights: [2.8769017041706473]
Loss: 15.2965775, Weights: [3.158547917811696]
Loss: 1.9145057, Weights: [3.5496563105844143]
Loss: 1.1350271, Weights: [3.75720480586606]
Loss:

In [254]:
neuron.forward_pass([20])        # Correct Answer :)

{'weighted_sum': 50.00000000608037, 'activation': 50.00000000608037}